## Detector charachterization (Figure 3)

We are characterizing the detector

In [18]:
# Import modules
import sys, h5py
import numpy as np
%matplotlib inline

# Import modules from src directory
sys.path.append("../src")
import plotting

In [19]:
# Gaussian model
gaussian = lambda x,A,mu,s: A * np.exp(-((mu-x)**2 /(2*s**2)))

In [20]:
# Pixel coordinates
bp = (8,238)
fp = (0,111)

### 1. Loading data from file

In [21]:
# Data files (back)
back_signal_histfile    = '../analysis/detector/back/signal/merged_histogram.h5'
back_signal_gainfile    = '../analysis/detector/back/signal/gainmap.h5'
back_signal_bgsigmafile = '../analysis/detector/back/signal/bg_sigmamap.h5'
back_signal_fittingfile = '../analysis/detector/back/signal/results.h5'
back_signal_maskfile    = '../analysis/detector/back/signal/included-pixels.h5'
back_dark_histfile      = '../analysis/detector/back/dark/histogram.h5'
back_dark_powderfile    = '../analysis/detector/back/dark/powder.h5'
back_geometryfile       = '../analysis/detector/back/back_geometry.h5'

# Data files (front)
front_signal_histfile    = '../analysis/detector/front/signal/merged_histogram.h5'
front_signal_gainfile    = '../analysis/detector/front/signal/gainmap.h5'
front_signal_bgsigmafile = '../analysis/detector/front/signal/bg_sigmamap.h5'
front_signal_fittingfile = '../analysis/detector/front/signal/results.h5'
front_signal_maskfile    = '../analysis/detector/front/signal/included-pixels.h5'
front_geometryfile       = '../analysis/detector/front/front_geometry.h5'
front_bad_pixels_file    = '../analysis/detector/front/front_bad-pixels.h5'

In [22]:
# Load histogram (back)
with h5py.File(back_signal_histfile, 'r') as f:
    back_ncount = float(f['data/histogramCount'][0])
    back_hist = f['data/histogram'][bp[0], bp[1]] / back_ncount
    hmin  = f['data/histogramMin'][:]
    hbins = f['data/histogramNbins'][:]
    hsize = f['data/histogramBinsize'][:]
    back_histbins = np.arange(hmin, hbins*hsize + hmin, hsize)

# Load dark histogram (back)
with h5py.File(back_dark_histfile, 'r') as f: 
    back_darkncount = float(f['data/histogramCount'][0])
    back_darkhist = f['data/histogram'][bp[0], bp[1]] / back_darkncount

# Load gain map (back)
with h5py.File(back_signal_gainfile, 'r') as f:
    back_gainmap = f['data/data'][:]

# Load bg sigma map (back)
with h5py.File(back_signal_bgsigmafile, 'r') as f:
    back_bgsigma = f['data/data'][:]

# Load fitting results (back)
with h5py.File(back_signal_fittingfile, 'r') as f:
    back_photon_amp   = f['photon_amp'][:].reshape(back_gainmap.shape) / back_ncount
    back_photon_mu    = f['photon_offset'][:].reshape(back_gainmap.shape)
    back_photon_sigma = f['photon_sigma'][:].reshape(back_gainmap.shape)
    back_bg_amp   = f['bg_amp'][:].reshape(back_gainmap.shape) / back_ncount
    back_bg_mu    = f['bg_offset'][:].reshape(back_gainmap.shape)
    back_bg_sigma = f['bg_sigma'][:].reshape(back_gainmap.shape)

# Load mask from gain fitting (back)
with h5py.File(back_signal_maskfile, 'r') as f:
    back_gainmask = f['data/data'][:].astype(np.bool)

# Load geometry (back)
with h5py.File(back_geometryfile, 'r') as f:
    x = np.floor(f['x']).astype(np.int)
    y = np.floor(f['y']).astype(np.int)
    x -= x.min()
    y -= y.min()
    back_shape = (y.max() - y.min() + 1, x.max() - x.min() + 1)
    back_x, back_y = x,y

# Load dark sigma (back)
with h5py.File(back_dark_powderfile, 'r') as f:
    back_darksigma = f['data/correcteddatasigma'][:]

In [23]:
# Load histogram (front)
with h5py.File(front_signal_histfile, 'r') as f:
    front_ncount = float(f['data/histogramCount'][0])
    front_hist = f['data/histogram'][fp[0], fp[1]] / front_ncount
    hmin  = f['data/histogramMin'][:]
    hbins = f['data/histogramNbins'][:]
    hsize = f['data/histogramBinsize'][:]
    front_histbins = np.arange(hmin, hbins*hsize + hmin, hsize)

# Load gain map (front)
with h5py.File(front_signal_gainfile, 'r') as f:
    front_gainmap = f['data/data'][:]

# Load bg sigma map (front)
with h5py.File(front_signal_bgsigmafile, 'r') as f:
    front_bgsigma = f['data/data'][:]

# Load fitting results (front)
with h5py.File(front_signal_fittingfile, 'r') as f:
    front_photon_amp   = f['photon_amp'][:].reshape(front_gainmap.shape) / front_ncount
    front_photon_mu    = f['photon_offset'][:].reshape(front_gainmap.shape)
    front_photon_sigma = f['photon_sigma'][:].reshape(front_gainmap.shape)
    front_bg_amp   = f['bg_amp'][:].reshape(front_gainmap.shape) / front_ncount
    front_bg_mu    = f['bg_offset'][:].reshape(front_gainmap.shape)
    front_bg_sigma = f['bg_sigma'][:].reshape(front_gainmap.shape)

# Load mask from gain fitting (front)
with h5py.File(front_signal_maskfile, 'r') as f:
    front_gainmask = f['data/data'][:].astype(np.bool)
    
# Load bad-pixel mask (front)
with h5py.File(front_bad_pixels_file, 'r') as f:
    front_badpixels = f['data/data'][:].astype(np.bool)
front_gainmask &= front_badpixels   

# Load geometry (front)
with h5py.File(front_geometryfile, 'r') as f:
    x = np.floor(f['x'][:]/(110*1e-6)).astype(np.int)
    y = np.floor(f['y'][:]/(110*1e-6)).astype(np.int)
    x -= x.min()
    y -= y.min()
    front_shape = (y.max() - y.min() + 1, x.max() - x.min() + 1)
    front_x, front_y = x,y

### 2. Assemble maps and masks

In [24]:
def assemble(raw, shape, x, y):
    assembled = np.zeros(shape).astype(raw.dtype)
    assembled[y,x] = raw
    return assembled
c = 700    

In [25]:
back_gainmap_a = assemble(back_gainmap, back_shape, back_x, back_y)                                                                                                               
back_bgsigma_a = assemble(back_bgsigma, back_shape, back_x, back_y)                                                                                                               
back_photon_amp_a = assemble(back_photon_amp, back_shape, back_x, back_y)                                                                                                         
back_bg_amp_a = assemble(back_bg_amp, back_shape, back_x, back_y)                                                                                                                 
back_mask_a = assemble(back_gainmask, back_shape, back_x, back_y)                                                                                                                 
#back_best_th_a = assemble(back_best_th, back_shape, back_x, back_y)                                                                                                               
front_gainmap_a = np.rot90(assemble(front_gainmap, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                                
front_bgsigma_a = np.rot90(assemble(front_bgsigma, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                                
front_photon_amp_a = np.rot90(assemble(front_photon_amp, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                          
front_bg_amp_a = np.rot90(assemble(front_bg_amp, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                                  
front_mask_a = np.rot90(assemble(front_gainmask, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                                  
front_badpix_a = np.rot90(assemble(front_badpixels, front_shape, front_x, front_y)[c:-c,c:-c], k=2)                                                                               
#front_best_th_a = np.rot90(assemble(front_best_th, front_shape, front_x, front_y)[c:-c,c:-c], k=2)

### 3. Fit Gaussians

In [26]:
# Compute gaussians (back)
back_gauss_0 = gaussian(back_histbins, back_bg_amp[bp[0],bp[1]], back_bg_mu[bp[0], bp[1]], back_bg_sigma[bp[0],bp[1]])
back_gauss_1 = gaussian(back_histbins, back_photon_amp[bp[0], bp[1]], back_photon_mu[bp[0],bp[1]], back_photon_sigma[bp[0],bp[1]])

In [27]:
# Compute gaussians (front)
front_gauss_0 = gaussian(front_histbins, front_bg_amp[fp[0],fp[1]], front_bg_mu[fp[0], fp[1]], front_bg_sigma[fp[0],fp[1]])
front_gauss_1 = gaussian(front_histbins, front_photon_amp[fp[0], fp[1]], front_photon_mu[fp[0],fp[1]], front_photon_sigma[fp[0],fp[1]])

### 4. Define Signal-to-noise ratio

In [28]:
# Select good pixels
backselect = (back_gainmap[back_gainmask] > 18) & (back_gainmap[back_gainmask] < 33) & (back_bgsigma[back_gainmask] > 3.2) & (back_bgsigma[back_gainmask] < 5.2)
frontselect = (front_gainmap_a[front_mask_a] > 9) & (front_gainmap_a[front_mask_a] < 24) & (front_bgsigma_a[front_mask_a] > 2) & (front_bgsigma_a[front_mask_a] < 4)

# Map of Signal to noise
back_smap = back_gainmap[back_gainmask][backselect] / back_bgsigma[back_gainmask][backselect]
front_smap = front_gainmap_a[front_mask_a][frontselect] / front_bgsigma_a[front_mask_a][frontselect]

# Map of signal above noise (smap) for back
back_smap_a = back_gainmap_a / back_bgsigma_a
back_smap_hist, back_smap_bins = np.histogram(back_smap, range=(0,10), bins=200, density=True)

# Map of signal above noise (smap) for front
front_smap_a = (front_gainmap_a / front_bgsigma_a)
front_smap_hist, front_smap_bins = np.histogram(front_smap, range=(0,10), bins=200, density=True)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in divide
/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in divide
/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in divide
/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in divide


### 5. Plotting

In [29]:
# Second idea for a plot
plot = plotting.Plot(rows=3, cols=4, exclude=range(16), fontsize=8, legend=True, legend_frameon=False, save_pdf=True)
plot.add_axes((0,2), 2,1, pady=0.075, hfrac=0.9)
plot.add_axes((2,2), 2,1, pady=0.075, hfrac=0.9)
plot.add_axes((0,1), 2,1, hfrac=0.9)
plot.add_axes((2,1), 2,1, hfrac=0.9)
plot.add_axes((0,0), 4,1, pady=-0.075, hfrac=0.9)

plot.xlabel = ['Detected signal [ADU]', 
               'Detected signal [ADU]', 
               'Noise parameter $\sigma^{(0)}$', 
               'Noise paramter $\sigma^{(0)}$',  
               r'Signal to noise ratio (SNR) $\mu^{(1)} - \mu^{(0)}$ / $\sigma^{(0)}$']
plot.ylabel = ['Probability density', 
               '', 
               r'Detector gain $\mu^{(1)} - \mu^{(0)}$', 
               'Detector gain $\mu^{(1)} - \mu^{(0)}$', 
               'Probability density']
plot.title_label = ['140k CSPAD (back)', '   2.3M CSPAD (front)', '', '', '']
plot.colorbar_label = r''

# Back detector
plot.plotting_traces(0, 3*[back_histbins],  [back_hist, back_gauss_0, back_gauss_1], 
                     ['Pixel histogram', 'Gaussian fit (0-photon)', 'Gaussian fit (1-photon)'],
                     colors=['k', 'b', 'r'], linewidths=[1,1,1], logy=True, ylim=[1e-6,1])
plot.axes[0].axvline((back_photon_mu[bp[0], bp[1]] - back_bg_mu[bp[0], bp[1]])*0.7, color='k', ls=':')
plot.axes[0].set_xticks(range(-20,100,20))
plot.axes[0].text(0, 1e-5, r'$\mu^{0}$, $\sigma^{0}$', color='b', ha='center')
plot.axes[0].text(55,1e-5, r'$\mu^{1}$, $\sigma^{1}$', color='r', ha='center')
plot.axes[0].text(-16,0.5, '(a)', fontsize=10, va='top', ha='left')

# Front detector
plot.plotting_traces(1, 3*[front_histbins], [front_hist, front_gauss_0, front_gauss_1], 
                     ['Pixel histogram', 'Gaussian fit (0-photon)', 'Gaussian fit (1-photon)'],
                     colors=['k', 'b', 'r'], linewidths=[1,1,1], logy=True, ylim=[1e-6,1])
plot.axes[1].axvline((front_photon_mu[fp[0], fp[1]] - front_bg_mu[fp[0], fp[1]])*0.7, color='k', ls=':')
plot.axes[1].set_yticklabels([])                                                                                                                                                  
plot.axes[1].text(-16,0.5, '(b)', fontsize=10, va='top', ha='left')

# Plot correlation
plot.legend_location = 4
plot.plotting_a_heatmap(2, back_bgsigma[back_gainmask], back_gainmap[back_gainmask], [[3.2,5.2],[18,33]], 
                        under='w', bins=100,alpha=1, cmaplist=['Greys'], zorder=0)
plot.plotting_traces(2, [np.linspace(3.2,5.2,100)], [6.*np.linspace(3.2,5.2,100)], ['SNR=6'], 
                     linestyles=['--'], colors=['b'], zorder=1)
plot.plotting_correlation(2, back_bgsigma[bp[0], bp[1]], back_gainmap[bp[0], bp[1]], color='r', 
                          ylim=[18,33], xlim=[3.2,5.2], zorder=2)
plot.axes[2].text(3.25,32, '(c)', fontsize=10, va='top', ha='left')

plot.plotting_a_heatmap(3, front_bgsigma_a[front_mask_a], front_gainmap_a[front_mask_a], [[2,4],[9,24]], 
                        under='w', bins=100,alpha=1, cmaplist=['Greys'], zorder=0)
plot.plotting_traces(3, [np.linspace(2,4,100)], [5.5*np.linspace(2,4,100)], ['SNR=5.5'], 
                     linestyles=['--'], colors=['b'], zorder=1)
plot.plotting_correlation(3, front_bgsigma[fp[0], fp[1]], front_gainmap[fp[0], fp[1]], color='r', 
                          ylim=[9,24], xlim=[2,4], zorder=2)
plot.legend_location = 1
plot.axes[3].text(2.05, 23, '(d)', fontsize=10, va='top', ha='left')
plot.axes[3].yaxis.set_label_position('right')
plot.axes[3].tick_params(axis='y', which='both', left='off', right='on', labelleft='off', labelright='on')

# Combined
plot.plotting_a_histogram(4, back_smap_hist, back_smap_bins, type='line', color='k', linestyle=':', 
                          edgewidth=1, logy=False, ylim=[0,2.5], label=['140k CSPAD (back)'])
plot.plotting_a_histogram(4, front_smap_hist, front_smap_bins, type='line', color='k', linestyle='--', 
                          edgewidth=1, logy=False, ylim=[0,2.5], label=['2.3M CSPAD (front)'])
plot.axes[4].text(0.1,2.4, '(e)', fontsize=10, va='top', ha='left')

# Save plot to file                                                                                                                                                               
plot.save('/Users/benedikt/phd-project/documentation/manuscripts/omrv-paper/manuscript/figures/fig_detectorchar.pdf')
plot.show()

**Figure 3.**
Pixel-wise characterization of the CSPAD detectors.                                                                                                                       
(a)-(b) Normalized histograms for two representative pixels integrated across all detected frames with Gaussian functions                                                          
fitted to the 0- and 1-photon peaks.                                                                                                                                               
(c)-(d) 2D histograms of noise and gain estimates placed on a grid of $100\times100$ pixels. The red dots correspond                                                               
to the values of $\mu^{(0)}$, $\mu^{(1)}$ and $\sigma^{(0)}$ shown in the pixel histograms above. $97.4$\% (back) and $85.1$\% (front) of all detector pixels                      
are represented, the rest are outliers due to misfitting.                                                                                                                          
(e) Signal-to-noise ratio for all valid pixels shown as a histogram.}

### 6. Find good pixels for showing histograms

In [59]:
back_point = np.where((back_gainmap > 25.5) & (back_gainmap < 25.6) & (back_bgsigma > 4.24) & (back_bgsigma < 4.25))                                                              
print "Back", back_point[0][:10], back_point[1][:10]                                                                                                                              
                                                                                                                                                                                  
front_point = np.where((front_gainmap > 16.65) & (front_gainmap < 16.75) & (front_bgsigma > 3.025) & (front_bgsigma < 3.035))                                                     
print "Front", front_point[0][:10], front_point[1][:10] 

Back [ 8 17 18 19 23 27 28 34 37 37] [238 311 303 308 378  45  65 271   9 267]
Front [ 0  9 11 17 17 19 20 21 23 24] [ 111  309  245  308 1158 1275  155 1520 1359  132]
